In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from openai import OpenAI

load_dotenv()

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL_NAME = "gpt-4o-mini"


In [2]:
df = pd.read_excel("サンプルデータ.xlsx", sheet_name="売上データ")
list(df.columns)


['カテゴリー', '商品コード', '商品名', '売上日', '単価', '数量', '原価']

In [3]:
list(df.columns)


['カテゴリー', '商品コード', '商品名', '売上日', '単価', '数量', '原価']

In [4]:
# 売上金額
df["売上金額"] = df["単価"] * df["数量"]

# 原価金額
df["原価金額"] = df["原価"] * df["数量"]

# 利益
df["利益"] = df["売上金額"] - df["原価金額"]

df.head()


,カテゴリー,商品コード,商品名,売上日,単価,数量,原価,売上金額,原価金額,利益
0,食品,1001,りんご,2023-01-01,200,50,120,10000,6000,4000
1,食品,1002,バナナ,2023-01-01,150,100,80,15000,8000,7000
2,食品,1003,牛乳,2023-01-02,180,80,100,14400,8000,6400
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800,30000,16000,14000
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500,50000,25000,25000


In [5]:
summary = (
    df.groupby(["カテゴリー", "商品コード", "商品名"], as_index=False)
    .agg(
        売上数量合計=("数量", "sum"),
        売上金額合計=("売上金額", "sum"),
        原価金額合計=("原価金額", "sum"),
        利益合計=("利益", "sum")
    )
)

summary["利益率"] = summary["利益合計"] / summary["売上金額合計"]
summary.head()


,カテゴリー,商品コード,商品名,売上数量合計,売上金額合計,原価金額合計,利益合計,利益率
0,日用品,3001,歯ブラシ,50,15000,5000,10000,0.666667
1,日用品,3002,シャンプー,30,24000,12000,12000,0.500000
2,日用品,3003,ティッシュペーパー,80,16000,8000,8000,0.500000
3,日用品,3004,洗剤,50,30000,15000,15000,0.500000
4,日用品,3005,柔軟剤,40,20000,10000,10000,0.500000


In [6]:
summary["利益率"] = pd.to_numeric(summary["利益率"], errors="coerce").fillna(0)
summary["利益率(%)"] = (summary["利益率"] * 100).round(1)


In [7]:
# 列の並び順を整理
summary = summary[
    [
        "カテゴリー",
        "商品コード",
        "商品名",
        "売上数量合計",
        "売上金額合計",
        "原価金額合計",
        "利益合計",
        "利益率(%)",
    ]
]

# 売上金額が高い順に並べ替え
summary = summary.sort_values("売上金額合計", ascending=False)

summary.head()


,カテゴリー,商品コード,商品名,売上数量合計,売上金額合計,原価金額合計,利益合計,利益率(%)
84,衣服,2008,スーツ,3,60000,30000,30000,50.0
97,衣服,2021,パーカー,15,52500,26250,26250,50.0
82,衣服,2006,コート,5,50000,25000,25000,50.0
78,衣服,2002,ジーンズ,10,50000,25000,25000,50.0
92,衣服,2016,スニーカー,10,50000,25000,25000,50.0


In [8]:
# 利益金額が高い順１０個に並べ替え
summary = summary.sort_values("利益合計", ascending=False)
summary.head(10)


,カテゴリー,商品コード,商品名,売上数量合計,売上金額合計,原価金額合計,利益合計,利益率(%)
84,衣服,2008,スーツ,3,60000,30000,30000,50.0
97,衣服,2021,パーカー,15,52500,26250,26250,50.0
82,衣服,2006,コート,5,50000,25000,25000,50.0
78,衣服,2002,ジーンズ,10,50000,25000,25000,50.0
92,衣服,2016,スニーカー,10,50000,25000,25000,50.0
81,衣服,2005,ワンピース,8,48000,24000,24000,50.0
101,衣服,2025,ジャケット,8,48000,24000,24000,50.0
80,衣服,2004,スカート,12,48000,24000,24000,50.0
126,衣服,2050,サングラス,15,45000,22500,22500,50.0
79,衣服,2003,セーター,15,45000,22500,22500,50.0
